### 전체 구조는 다 짰음. 그러나 g(x)의 값이 왜 이렇게 크게 나오는지 의문. 추후 해결해야함. 

# Anomaly Detection - MOG


1. 관측 데이터가 다수의 가우시안 분포로부터 생성되었다고 가정하여 확률 밀도 함수 도출하기 

2. E / M 알고리즘을 통해서 객체 확률 / 가우시안 파라미터들의 값을 각각 최적화하기 

3. 적절한 가우시안 분포 개수 정하기 => class 밖에서 확인. 

**구현해야 하는 것**
- $g(x|\mu_m, \sum_m) = \frac{1}{(2\pi)^{d/2} |\sum_m|^{\frac{1}{2}}}exp(\frac{1}{2}(x-\mu_m)^T\sum_m^{-1}(x-\mu_m))$
- $p(x|\lambda) = \sum_{m=1}^Mw_mg(x|\mu_m, \sum_m)$

- 임의의 파라미터 값들을 부여하기 

- E Step : 가우시안 파라미터를 고정하여 객체 확률 값 계산 
> $p(m|x_i, \lambda) = \frac{w_mg(x_i|\mu_m, \sum_m)}{\sum_{k=1}^M w_mg(x_i|\mu_m, \sum_m)}$

- M Step : 객체 확률 값을 고정하여 가우시안 파라미터 값들 계산하기 

> $w_m^{new} = \frac{1}{N} \sum_{i=1}^N p(m|x_i, \lambda)$ 

> $\mu_m^{new} = \frac{\sum_{i=1}^N p(m|x_i, \lambda)x_i}{\sum_{i=1}^Np(m|x_i, \lambda)}$

> $\sigma_m^{2(new)} = \frac{\sum_{i=1}^N p(m|x_i, \lambda)x_i^2}{\sum_{i=1}^N p(m|x_i,\lambda)} - \mu_m^{2(new)}$



**필요한 것**
- X 
- d 
- m
- $\lambda = {w_m, \mu_m, \sum_M}$ 
- Var_type : Spherical / diagonal / full 
>  이건 이번에 고려 X. 기본적으로 Diagonal을 가정 



**함수의 형태**
- def __init__(self, X, Var_type) : 
  
- def lambda(self) : => m개의 [w_1, $\mu_1, \sum_1$] list를 포괄한 list제작 
> 분산의 형태에 대해서 지정해줄 수 있어야 함. 

- def expectation(self) : self.lambda를 입력값으로 받아 self.p 값 업데이트 
- def maximization(self) : self.p 를 입력값으로 받아 self.lambda 값 업데이트 

- def sol(self) : E/M 과정의 답이 오차 내로 줄어들때까지 진행. 최후 우도값으로 반환할 수 있을 것. 


In [3]:
import numpy as np
import pandas as pd
import random as rand

from sklearn.datasets import load_iris
X = load_iris()['data']

import matplotlib.pyplot as plt
import scipy as sc
from scipy.stats import norm
from sys import maxsize

In [66]:
class MOG() : 
    def __init__(self, X, m) : 
        self.X = np.array(X) 
        self.n = np.shape(X)[0] 
        self.d = np.shape(X)[1]
        
        self.m = m 
        
        self.w, self.mu, self.sigma, self.cov = self.set_parameter()
        self.p = self.expectation()
        
    def set_parameter(self) : 
        # w 는 m분의 1로 균등하게 부여 
        w = np.ones(self.m) / self.m 
        
        # mu는 각 변수들의 최소 ~ 최대 값 사이의 랜덤한 값으로 부여 
        min_value = np.apply_along_axis(lambda a : np.min(a), 0, self.X)
        max_value = np.apply_along_axis(lambda a : np.max(a), 0, self.X)
        mu = [] 
        sigma = [] 
        for i in range(self.m) : 
            np.random.seed(i)
            mu.append([np.random.uniform(max_value[j], min_value[j]) for j in range(self.d)])
            sigma.append([np.random.uniform(max_value[j] - mu[i][j], mu[i][j] -min_value[j]) for j in range(self.d)])    
        mu = np.array(mu)
        sigma = np.array(sigma)
        cov = [np.diag(sigma[i]) for i in range(self.m)] 
        
        return w, mu, sigma, cov

    
    def g(self, x, m) : 
        x = np.array(x)
        inv_cov = np.linalg.inv(self.cov[m])
        upper_value =  np.exp((x-self.mu[m])@inv_cov@(x-self.mu[m]).T /2)
        under_value = ((2*np.pi)**(self.d/2)) * np.sqrt(np.sqrt((self.cov[m]**2).sum())) 
        print(upper_value)
        print(under_value)
        return upper_value / under_value

    def expectation(self): 
        p = np.zeros((self.n, self.m))
        
        for i in range(self.n) : 
            vector = np.array([self.g(self.X[i], j) for j in range(self.m)]) 
            p[i] = vector / vector.sum()
        return p # m x n 행렬 
    
    def maximization(self) :
        w = self.p.sum(axis= 0) / self.n
        mu = np.array([(self.X * self.p[:,j].reshape(-1,1)).sum(axis=0) / self.p[:,j].sum(axis=0) for j in range(self.m) ]) 
        sigma = np.array([(self.X**2 * self.p[:,j].reshape(-1,1)).sum(axis=0) / self.p[:,j].sum(axis=0) - mu[j]**2 for j in range(self.m) ]) 
        cov = np.array([np.diag(sigma[i]) for i in range(self.m)]) 
        return w, mu, sigma, cov 
        
    def find_parameter(self, epsilon = 1e-40) : 
        pre = self.p[:,:] 
        self.p = self.expectation() 
        self.w, self.mu, self.sigma, self.cov = self.maximization() 
        diff = pre - self.p
        while diff.sum() > epsilon : 
            pre = self.p[:,:]
            self.p = self.expectation() 
            self.w, self.mu, self.sigma, self.cov = self.maximization()
            diff = pre - self.p 
        
        return self.p, self.w, self.mu, self.sigma, self.cov
    
    def check_abnormal(self,x, epsilon = 1e-5) : 
        value = np.array([self.w[i] * self.g(x,i) for i in range(self.m)]).sum()
        if value < epsilon : 
            return print("abnormal data. p:" ,value)
        else : 
            return print("normal data. p:", value)
        

In [67]:
test = MOG(X, 3)
test.find_parameter()

4.802836639828062
77.48205398208137
6403880.41968083
65.34000444908511
10.103121877113743
77.76497645249887
4.0571313869026575
77.48205398208137
6395165.341271616
65.34000444908511
22.919326135474293
77.76497645249887
5.3059567612767795
77.48205398208137
13655251.964935983
65.34000444908511
21.65599140872757
77.76497645249887
4.7857829768959785
77.48205398208137
5443191.479577578
65.34000444908511
23.85464003774835
77.76497645249887
5.471966759261601
77.48205398208137
7357061.531039272
65.34000444908511
9.861822268582568
77.76497645249887
4.518338331406184
77.48205398208137
1164902.0140700042
65.34000444908511
4.1637728680630826
77.76497645249887
5.505371152562335
77.48205398208137
8600279.376543626
65.34000444908511
15.553341785864072
77.76497645249887
4.399243105209312
77.48205398208137
4048546.4942320073
65.34000444908511
11.289427670477236
77.76497645249887
5.606059503468645
77.48205398208137
10899578.939194107
65.34000444908511
43.77153672902095
77.76497645249887
4.332914346842057

5.326480629038173
77.48205398208137
9201811.945135092
65.34000444908511
22.12509666654171
77.76497645249887
4.962973860921102
77.48205398208137
3759261.2195775234
65.34000444908511
6.907936620079117
77.76497645249887
4.395169059537042
77.48205398208137
6375267.046040651
65.34000444908511
13.67944858355513
77.76497645249887
2.1536119935679383
77.48205398208137
11.77453959229288
65.34000444908511
2.9975023176605573
77.76497645249887
1.5721230215207953
77.48205398208137
15.482091137015097
65.34000444908511
2.487329128065276
77.76497645249887
2.194401106912948
77.48205398208137
7.309541841745874
65.34000444908511
3.601498962600722
77.76497645249887
1.2162169001944783
77.48205398208137
68.47010127464749
65.34000444908511
16.03977186218567
77.76497645249887
1.492481179485612
77.48205398208137
11.571964485568078
65.34000444908511
4.97306215569925
77.76497645249887
1.2793824041892634
77.48205398208137
17.88143861071536
65.34000444908511
5.413578076442104
77.76497645249887
1.8188683993726544
77

(array([[6.32458139e-07, 9.99998042e-01, 1.32558214e-06],
        [5.34987691e-07, 9.99996454e-01, 3.01122801e-06],
        [3.27673487e-07, 9.99998340e-01, 1.33251696e-06],
        [7.41439129e-07, 9.99995576e-01, 3.68224303e-06],
        [6.27214855e-07, 9.99998247e-01, 1.12628198e-06],
        [3.27088038e-06, 9.99993726e-01, 3.00323990e-06],
        [5.39822751e-07, 9.99997941e-01, 1.51951635e-06],
        [9.16337479e-07, 9.99996741e-01, 2.34296874e-06],
        [4.33734894e-07, 9.99996192e-01, 3.37423662e-06],
        [7.85461607e-07, 9.99995555e-01, 3.65917606e-06],
        [1.15246518e-06, 9.99997287e-01, 1.56011330e-06],
        [1.30138079e-06, 9.99995229e-01, 3.46994652e-06],
        [4.84352870e-07, 9.99996686e-01, 2.82965543e-06],
        [1.03874112e-07, 9.99999189e-01, 7.06706278e-07],
        [3.95345003e-07, 9.99999306e-01, 2.98440443e-07],
        [1.97486420e-06, 9.99997265e-01, 7.60452372e-07],
        [5.85240456e-07, 9.99998860e-01, 5.55117769e-07],
        [6.717

In [69]:
test.g([5.1, 3.5, 1.4, 0.2], 0)

3.7588950519176634e+17
30.173659145473632


1.2457537992973376e+16

In [63]:
a = np.array([[1,2]])
b = np.array([[3,4],[4,3]])
c = np.array([[2,3]])

b[1,1]

3

In [61]:
np.sqrt((b**2).sum())

7.0710678118654755

In [4]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]]) 
b = np.array([[1],[2],[3]])

a*b

array([[ 1,  2,  3],
       [ 8, 10, 12],
       [21, 24, 27]])